# Compute the Time of Emergence 

In [1]:
import os
import shutil
from itertools import product

import numpy as np
import xarray as xr

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pop_tools
import xpersist as xp

import dask

import util

Cannot write to data cache '/glade/p/cesmdata/cseg'. Will not be able to download remote data files. Use environment variable 'CESMDATAROOT' to specify another directory.


## Spin up cluster

In [2]:
from ncar_jobqueue import NCARCluster
from dask.distributed import Client
try:
    cluster
    client
except:
    cluster = NCARCluster()
    cluster.scale(30)
    client = Client(cluster) # Connect this local process to remote workers
client

Client Scheduler: tcp://10.12.205.15:37648 Dashboard: https://jupyterhub.ucar.edu/dav/user/mclong/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


## Load DJF dataset

In [3]:
ds_djf = xr.open_zarr(f'{util.project_tmpdir}/cesm-le-fields-djf.zarr')
ds_djf = ds_djf[['SST', 'Chl_surf', 'KGP']]
ds_djf

<xarray.Dataset>
Dimensions:    (length: 3, member_id: 34, nlat: 93, nlon: 320, time: 180)
Coordinates:
    TLAT       (nlat, nlon) float64 dask.array<chunksize=(93, 320), meta=np.ndarray>
    TAREA      (nlat, nlon) float64 dask.array<chunksize=(93, 320), meta=np.ndarray>
    ULONG      (nlat, nlon) float64 dask.array<chunksize=(93, 320), meta=np.ndarray>
    ULAT       (nlat, nlon) float64 dask.array<chunksize=(93, 320), meta=np.ndarray>
  * length     (length) float64 20.0 40.0 60.0
    KMT        (nlat, nlon) float64 dask.array<chunksize=(93, 320), meta=np.ndarray>
  * time       (time) int64 1921 1922 1923 1924 1925 ... 2097 2098 2099 2100
    TLONG      (nlat, nlon) float64 dask.array<chunksize=(93, 320), meta=np.ndarray>
  * member_id  (member_id) int64 1 2 9 10 11 12 13 ... 34 35 101 102 103 104 105
Dimensions without coordinates: nlat, nlon
Data variables:
    SST        (time, member_id, nlat, nlon) float64 dask.array<chunksize=(180, 1, 93, 320), meta=np.ndarray>
    Chl_surf   (time, member_id, nlat, nlon) float64 dask.array<chunksize=(180, 1, 93, 320), meta=np.ndarray>
    KGP        (time, length, member_id, nlat, nlon) float64 dask.array<chunksize=(180, 3, 1, 93, 320), meta=np.ndarray>

## Compute the properties of the unperturbed climate

The mean

In [4]:
%%time
ds_ctrl_mean = (
        ds_djf.sel(time=slice(1920, 1950))
        .mean(dim=['time', 'member_id'])
        .compute()
)
    
ds_ctrl_mean

CPU times: user 401 ms, sys: 33.6 ms, total: 435 ms
Wall time: 5.55 s


<xarray.Dataset>
Dimensions:   (length: 3, nlat: 93, nlon: 320)
Coordinates:
    TLAT      (nlat, nlon) float64 -79.22 -79.22 -79.22 ... -30.08 -30.08 -30.08
    TAREA     (nlat, nlon) float64 1.125e+13 1.125e+13 ... 6.346e+13 6.346e+13
    ULONG     (nlat, nlon) float64 321.1 322.3 323.4 324.5 ... 317.8 318.9 320.0
    ULAT      (nlat, nlon) float64 -78.95 -78.95 -78.95 ... -29.82 -29.82 -29.82
  * length    (length) float64 20.0 40.0 60.0
    KMT       (nlat, nlon) float64 0.0 0.0 0.0 0.0 0.0 ... 53.0 53.0 53.0 54.0
    TLONG     (nlat, nlon) float64 320.6 321.7 322.8 323.9 ... 317.2 318.3 319.4
Dimensions without coordinates: nlat, nlon
Data variables:
    SST       (nlat, nlon) float64 0.0 0.0 0.0 0.0 ... 23.14 23.04 22.99 22.96
    Chl_surf  (nlat, nlon) float64 0.0 0.0 0.0 0.0 ... 0.1035 0.1016 0.09941
    KGP       (length, nlat, nlon) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0

Standard deviation of anomalies

In [5]:
%%time
ds_ctrl_std = (
        (ds_djf - ds_djf.mean(dim=['member_id']).sel(time=slice(1920, 1950)))
        .std(dim=['time', 'member_id'])
        .compute()
)
ds_ctrl_std

CPU times: user 971 ms, sys: 92.7 ms, total: 1.06 s
Wall time: 12.5 s


<xarray.Dataset>
Dimensions:   (length: 3, nlat: 93, nlon: 320)
Coordinates:
    TLAT      (nlat, nlon) float64 -79.22 -79.22 -79.22 ... -30.08 -30.08 -30.08
    TAREA     (nlat, nlon) float64 1.125e+13 1.125e+13 ... 6.346e+13 6.346e+13
    ULONG     (nlat, nlon) float64 321.1 322.3 323.4 324.5 ... 317.8 318.9 320.0
    ULAT      (nlat, nlon) float64 -78.95 -78.95 -78.95 ... -29.82 -29.82 -29.82
  * length    (length) float64 20.0 40.0 60.0
    KMT       (nlat, nlon) float64 0.0 0.0 0.0 0.0 0.0 ... 53.0 53.0 53.0 54.0
    TLONG     (nlat, nlon) float64 320.6 321.7 322.8 323.9 ... 317.2 318.3 319.4
Dimensions without coordinates: nlat, nlon
Data variables:
    SST       (nlat, nlon) float64 0.0 0.0 0.0 0.0 ... 0.4305 0.4294 0.4313
    Chl_surf  (nlat, nlon) float64 0.0 0.0 0.0 ... 0.002909 0.003036 0.003102
    KGP       (length, nlat, nlon) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0

## Compute the sign of the forced trend

In [6]:
%%time
ds_trend_sign = xr.where(
    ds_djf.sel(time=slice(2091, 2100)).mean(['time', 'member_id']) >= ds_ctrl_mean, 1., -1
).compute()
ds_trend_sign    

CPU times: user 432 ms, sys: 28.3 ms, total: 460 ms
Wall time: 1.13 s


<xarray.Dataset>
Dimensions:   (length: 3, nlat: 93, nlon: 320)
Coordinates:
    TLAT      (nlat, nlon) float64 -79.22 -79.22 -79.22 ... -30.08 -30.08 -30.08
    TAREA     (nlat, nlon) float64 1.125e+13 1.125e+13 ... 6.346e+13 6.346e+13
    ULONG     (nlat, nlon) float64 321.1 322.3 323.4 324.5 ... 317.8 318.9 320.0
    ULAT      (nlat, nlon) float64 -78.95 -78.95 -78.95 ... -29.82 -29.82 -29.82
  * length    (length) float64 20.0 40.0 60.0
    KMT       (nlat, nlon) float64 0.0 0.0 0.0 0.0 0.0 ... 53.0 53.0 53.0 54.0
    TLONG     (nlat, nlon) float64 320.6 321.7 322.8 323.9 ... 317.2 318.3 319.4
Dimensions without coordinates: nlat, nlon
Data variables:
    SST       (nlat, nlon) float64 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0
    Chl_surf  (nlat, nlon) float64 1.0 1.0 1.0 1.0 1.0 ... -1.0 -1.0 -1.0 -1.0
    KGP       (length, nlat, nlon) float64 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0

## Compute the value of the threshold for ToE

In [7]:
ds_toe_threshold = ds_ctrl_mean + 2. * ds_ctrl_std * ds_trend_sign
ds_toe_threshold

<xarray.Dataset>
Dimensions:   (length: 3, nlat: 93, nlon: 320)
Coordinates:
    TLAT      (nlat, nlon) float64 -79.22 -79.22 -79.22 ... -30.08 -30.08 -30.08
    TAREA     (nlat, nlon) float64 1.125e+13 1.125e+13 ... 6.346e+13 6.346e+13
    ULONG     (nlat, nlon) float64 321.1 322.3 323.4 324.5 ... 317.8 318.9 320.0
    ULAT      (nlat, nlon) float64 -78.95 -78.95 -78.95 ... -29.82 -29.82 -29.82
  * length    (length) float64 20.0 40.0 60.0
    KMT       (nlat, nlon) float64 0.0 0.0 0.0 0.0 0.0 ... 53.0 53.0 53.0 54.0
    TLONG     (nlat, nlon) float64 320.6 321.7 322.8 323.9 ... 317.2 318.3 319.4
Dimensions without coordinates: nlat, nlon
Data variables:
    SST       (nlat, nlon) float64 0.0 0.0 0.0 0.0 0.0 ... 24.0 23.9 23.85 23.82
    Chl_surf  (nlat, nlon) float64 0.0 0.0 0.0 0.0 ... 0.0977 0.09556 0.0932
    KGP       (length, nlat, nlon) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0

Quick plot

## Compute the time-evolving 2$\sigma$ timeseries

In [8]:
%%time
ds_2sigma = ds_djf.mean('member_id') + 2. * ds_trend_sign * ds_djf.std('member_id')
ds_2sigma = ds_2sigma.compute()
ds_2sigma

CPU times: user 2.13 s, sys: 445 ms, total: 2.58 s
Wall time: 26.5 s


<xarray.Dataset>
Dimensions:   (length: 3, nlat: 93, nlon: 320, time: 180)
Coordinates:
    TLAT      (nlat, nlon) float64 -79.22 -79.22 -79.22 ... -30.08 -30.08 -30.08
    TAREA     (nlat, nlon) float64 1.125e+13 1.125e+13 ... 6.346e+13 6.346e+13
    ULONG     (nlat, nlon) float64 321.1 322.3 323.4 324.5 ... 317.8 318.9 320.0
    ULAT      (nlat, nlon) float64 -78.95 -78.95 -78.95 ... -29.82 -29.82 -29.82
  * length    (length) float64 20.0 40.0 60.0
    KMT       (nlat, nlon) float64 0.0 0.0 0.0 0.0 0.0 ... 53.0 53.0 53.0 54.0
  * time      (time) int64 1921 1922 1923 1924 1925 ... 2096 2097 2098 2099 2100
    TLONG     (nlat, nlon) float64 320.6 321.7 322.8 323.9 ... 317.2 318.3 319.4
Dimensions without coordinates: nlat, nlon
Data variables:
    SST       (time, nlat, nlon) float64 0.0 0.0 0.0 0.0 ... 27.12 27.05 26.99
    Chl_surf  (time, nlat, nlon) float64 0.0 0.0 0.0 ... 0.06238 0.06209 0.06176
    KGP       (time, length, nlat, nlon) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0

## Identify where the 2 sigma threshold has been crossed

In [9]:
%%time
ds_x_thresh = xr.where(ds_trend_sign > 0,
               ds_2sigma > ds_toe_threshold,
               ds_2sigma < ds_toe_threshold,               
               ).compute()
ds_x_thresh

CPU times: user 86.3 ms, sys: 25.4 ms, total: 112 ms
Wall time: 100 ms


<xarray.Dataset>
Dimensions:   (length: 3, nlat: 93, nlon: 320, time: 180)
Coordinates:
    KMT       (nlat, nlon) float64 0.0 0.0 0.0 0.0 0.0 ... 53.0 53.0 53.0 54.0
    TLONG     (nlat, nlon) float64 320.6 321.7 322.8 323.9 ... 317.2 318.3 319.4
    TLAT      (nlat, nlon) float64 -79.22 -79.22 -79.22 ... -30.08 -30.08 -30.08
    TAREA     (nlat, nlon) float64 1.125e+13 1.125e+13 ... 6.346e+13 6.346e+13
    ULONG     (nlat, nlon) float64 321.1 322.3 323.4 324.5 ... 317.8 318.9 320.0
    ULAT      (nlat, nlon) float64 -78.95 -78.95 -78.95 ... -29.82 -29.82 -29.82
  * length    (length) float64 20.0 40.0 60.0
  * time      (time) int64 1921 1922 1923 1924 1925 ... 2096 2097 2098 2099 2100
Dimensions without coordinates: nlat, nlon
Data variables:
    SST       (nlat, nlon, time) bool False False False False ... True True True
    Chl_surf  (nlat, nlon, time) bool False False False False ... True True True
    KGP       (length, nlat, nlon, time) bool False False False ... False False

## Compute ToE

In [ ]:
%%time

@dask.delayed
def assess_toe(da_x_thresh):
    """Function to compute ToE from boolean threshold array"""
    if not any(da_x_thresh[:]):
        return da_x_thresh.time.data[-1]
    else:
        for l in range(len(da_x_thresh.time)):
             if all(da_x_thresh[l:]):
                return da_x_thresh.time.data[l]
        return da_x_thresh.time.data[-1]
    
ds_toe = xr.Dataset()
for v in ['SST', 'Chl_surf', 'KGP']:
    # stack lateral dims
    da = ds_x_thresh[v].stack(non_time_dims=[d for d in ds_x_thresh[v].dims if d != 'time'])
    
    # initialize temporary variable
    da_toe_stack = da.isel(time=0, drop=True)

    # loop over non time dims; compute ToE
    toe_results = []
    for i in range(len(da.non_time_dims)):
        toe_results.append(assess_toe(da[:, i]))        
    da_toe_stack.data = np.array(dask.compute(*toe_results)).astype(da.time.dtype)
    
    # assign to output dataset
    ds_toe[v] = da_toe_stack.unstack('non_time_dims')
    
ds_toe = ds_toe.where(ds_toe.KMT > 0)

ds_toe

## Illustrative visualization

Did everything work?

- Dashed red lines show ±2$\sigma$ around unperturbed climate mean (dashed black)
- Heavy dashed red line shows the 2$\sigma$ value relevant as ToE threshold (depends on sign of change)
- Blue line shows the time-evolving 2$\sigma$ above (or below) the ensemble mean
- Star shows ToE: the first year where blue line is outside of red and stays out

In [ ]:
i = 100
j = 40
variables = [
    'SST',
    'Chl_surf',     
]

nrow = len(variables) 
ncol = 1
fig, axs = plt.subplots(nrow, ncol, figsize=(6*ncol, 4*nrow), squeeze=False) 

for n, v in enumerate(variables):
    ax = axs[np.unravel_index(n, axs.shape)]
    
    # plot the ensemble
    for m_id in ds_djf.member_id.values:
        ds_djf[v].isel(nlat=j, nlon=i).sel(member_id=m_id).plot(ax=ax, color='gray', alpha=0.5)

    ds_djf[v].isel(nlat=j, nlon=i).mean('member_id').plot(ax=ax, color='k')
    
    # plot the control climate ± std
    ax.axhline(ds_ctrl_mean[v].isel(nlat=j, nlon=i), lw=1, color='k', linestyle='--')
    ax.axhline(ds_ctrl_mean[v].isel(nlat=j, nlon=i) + 2 * ds_ctrl_std[v].isel(nlat=j, nlon=i), 
               lw=1, color='r', linestyle='--')
    ax.axhline(ds_ctrl_mean[v].isel(nlat=j, nlon=i) - 2 * ds_ctrl_std[v].isel(nlat=j, nlon=i), 
               lw=1, color='r', linestyle='--')
    
    # highlight the threshold 
    ax.axhline(ds_toe_threshold[v].isel(nlat=j, nlon=i), 
               lw=3, color='r', linestyle='--',               
              )
    
    ds_extreme[v].isel(nlat=j, nlon=i).plot(ax=ax, color='b', marker='.')
    
    ax.plot(
        ds_toe[v].isel(nlat=j, nlon=i), 
        ds_extreme[v].isel(nlat=j, nlon=i).sel(time=ds_toe[v].isel(nlat=j, nlon=i)), 
        'r*', 
        markersize=15
    )
    ax.set_title(v)

In [ ]:
ds_toe.Chl_surf.plot()

In [ ]:
ds_toe.SST.plot()

In [ ]:
ds_toe.KGP.sel(length=40).plot()

In [ ]:
util.write_ds_out(ds_toe, f'data/cesm-le-fields-djf-ToE-maps.zarr')

In [ ]:
client.close()
cluster.close()
del client
del cluster